In [1]:
import os
import psycopg2
import numpy as np
# Adding MaskedColumn - AEE
from astropy.table import Table, vstack, Column, MaskedColumn
# import argparse
# import time
# start_time = time.time()
import re
from datetime import date, timedelta
import time
start_time = time.time()

from desimeter.util import parse_fibers
from desimeter.dbutil import dbquery,get_petal_ids,get_pos_ids,get_petal_loc

In [2]:
# Configuration
date_to_analyze = 20210605
write_to_file = False
write_path = '/n/home/msdos/ann'
write_to_db = True
def output_to_file(the_table, output_filename, format='ascii.csv'):
    if format == 'ascii.csv':
        the_table.write(os.path.join(write_path, output_filename), format=format, overwrite=True)
    elif format == 'pandas.json':
        the_table.write(os.path.join(write_path, output_filename), format=format, indent=4, orient='records')
    else:
        the_table.write(os.path.join(write_path, output_filename), format=format)

In [3]:
# Commented this out - AEE
#import matplotlib.pyplot as plt
#%pylab inline

In [4]:
# The petal DB tables are by index number, but we refer to fibers by location
petal_id2loc = {4:0, 5:1, 6:2, 3:3, 8:4, 10:5, 11:6, 2:7, 7:8, 9:9}

In [5]:
# Connect to the database
args = {'host': os.getenv('DOS_DB_HOST'), 'port': int(os.getenv('DOS_DB_PORT')), 'database': os.getenv('DOS_DB_NAME'), 'user': os.getenv('DOS_DB_READER'), 'password': os.getenv('DOS_DB_READER_PASSWORD')}
print(args)

comm = psycopg2.connect(**args)

{'host': '', 'port': 1234, 'database': '', 'user': '', 'password': ''}


In [6]:
# Fetch a positioner's worth of data
# cmd = "select * from posmovedb.positioner_moves where (pos_id='M04866') and (time_recorded > '2021-05-17' ) order by time_recorded desc"
# db=dbquery(comm,cmd)
# db = Table(db)
# print(len(db), "entries retrieved")

In [7]:
# Fetch a petals's worth of data
# To select the data from a night, use the date of the next day.  So May 17 becomes 2021-05-18.
# cmd = "select * from posmovedb.positioner_moves_p8 where (time_recorded > '2021-05-19') order by time_recorded desc"
# db=dbquery(comm,cmd)
# db = Table(db)
# print(len(db), "entries retrieved")

In [8]:
# Fetch all petals's worth of data
# To select the data from a night, use the date of the next day.  So May 17 becomes 2021-05-18.
# TODO: Need to make the date handling better
def fetchDB(night):
    _night = f'{night:08d}'
    _night = date(int(_night[0:4]),int(_night[4:6]),int(_night[6:8]))
    today = _night+timedelta(days=1)
    tomorrow = _night+timedelta(days=2)
    print("Searching DB from", today.isoformat(), 'to', tomorrow.isoformat())
    dbs = {}
    for petal in petal_id2loc.keys():
        print("Fetching petal", petal)
        cmd = "select * from posmovedb.positioner_moves_p"+f'{petal:1d}'+" where (time_recorded > '"+today.isoformat()+"') and (time_recorded < '"+tomorrow.isoformat()+"') order by time_recorded desc"
    #    print(cmd)
        db=dbquery(comm,cmd)
        db = Table(db)
        dbs[petal]=db
        print(len(db), "entries retrieved")
    print("All petals retrieved.")
    return dbs


In [9]:
dbs = fetchDB(date_to_analyze)

Searching DB from 2021-06-06 to 2021-06-07
Fetching petal 4
21328 entries retrieved
Fetching petal 5
21405 entries retrieved
Fetching petal 6
21005 entries retrieved
Fetching petal 3
21143 entries retrieved
Fetching petal 8
21201 entries retrieved
Fetching petal 10
21189 entries retrieved
Fetching petal 11
21000 entries retrieved
Fetching petal 2
21212 entries retrieved
Fetching petal 7
21644 entries retrieved
Fetching petal 9
21666 entries retrieved
All petals retrieved.


In [10]:
print(dbs[3].dtype)

[('petal_id', '<i8'), ('device_loc', '<i8'), ('pos_id', '<U6'), ('pos_move_index', '<i8'), ('time_recorded', 'O'), ('bus_id', '<U5'), ('pos_t', '<f8'), ('pos_p', '<f8'), ('last_meas_obs_x', 'O'), ('last_meas_obs_y', 'O'), ('last_meas_peak', 'O'), ('last_meas_fwhm', 'O'), ('total_move_sequences', '<i8'), ('total_cruise_moves_t', '<i8'), ('total_cruise_moves_p', '<i8'), ('total_creep_moves_t', '<i8'), ('total_creep_moves_p', '<i8'), ('ctrl_enabled', '?'), ('move_cmd', '<U85'), ('move_val1', '<U129'), ('move_val2', '<U126'), ('log_note', '<U206'), ('exposure_id', 'O'), ('exposure_iter', 'O'), ('flags', 'O'), ('obs_x', 'O'), ('obs_y', 'O'), ('ptl_x', 'O'), ('ptl_y', 'O'), ('ptl_z', 'O'), ('site', '<U4'), ('postscript', 'O')]


Here's the main analysis code.

In [11]:
def analyze_fiber(db, quiet=False, showlog=False):
    # We have to parse some columns that should be integers, but aren't in the raw table.
    def parse_int(string):
        if string == None: return 0
        else: return int(string)
    def parse_float(string):
        if string == None: return 0
        else: return float(string)
    exposure = np.array(list(map(parse_int,db['exposure_id'])))
    flags = np.array(list(map(parse_int,db['flags'])))
    db_iter = np.array(list(map(parse_int,db['exposure_iter'])))
    # These two lines are new - AEE
    obs_x = np.array(list(map(parse_float,db['obs_x'])))
    obs_y = np.array(list(map(parse_float,db['obs_y'])))


    # A move was commanded
    move_attempted = db['ctrl_enabled'] & np.array(list(map(lambda s: (s.find('QS')>=0), db['move_cmd'])))
    # Which was supposed to be these computed angular distances:
    def sum_move(string):
        return np.sum(np.array(list(map(float, re.split('; | ',string)[1::2]))))
    move_t = np.array(list(map(sum_move, db['move_val1'])))
    move_p = np.array(list(map(sum_move, db['move_val2'])))
    # We care about big moves, more than 10 degrees.
    bigmove_t = move_attempted & (np.abs(move_t)>10)
    bigmove_p = move_attempted & (np.abs(move_p)>10)
    bigmove = bigmove_t|bigmove_p
    
    # Look for the commanded position
    re_prog = re.compile('req_posintTP=\(([\d\.-]*), ([\d\.-]*)\)')
    def parse_move(log):
        match = re_prog.search(log)
        if match: return np.array(list(map(float,match.groups())))
        else: return np.array([999.0,999.0])
    requested_moves = np.array(list(map(parse_move, db['log_note'])))
    attempted_move = (np.abs(np.where(requested_moves[:,0]<998.0, requested_moves[:,0]-db['pos_t'], 0.0))<0.01) & \
        (np.abs(np.where(requested_moves[:,1]<998.0, requested_moves[:,1]-db['pos_p'], 0.0))<0.01)
    
    # Look for interesting phrases in the logs
    # Make an array for lines containing handle_fvc_feedback and not containing reject
    fvc_feedback = db['ctrl_enabled'] & np.array(list(map(lambda s: (s.find('handle_fvc_feedback')>=0), db['log_note']))) \
        & np.array(list(map(lambda s: (s.find('reject')<0), db['log_note'])))
    interfere = np.array(list(map(lambda s: (s.find('interferes')>=0), db['log_note'])))
    debounced = np.array(list(map(lambda s: (s.find('Debounced')>=0), db['log_note'])))
    debounced_fail = np.array(list(map(lambda s: (s.find('debouncing polygons')>=0), db['log_note'])))
    frozen = np.array(list(map(lambda s: (s.find('freeze')>=0), db['log_note']))) & db['ctrl_enabled'] & ~attempted_move
    frozen_unflagged = np.array(list(map(lambda s: (s.find('freeze')>=0), db['log_note']))) & (flags<2**16) & (db_iter==0) & ~attempted_move
    denied = np.array(list(map(lambda s: (s.find('denied')>=0), db['log_note'])))
    autodisabled = np.array(list(map(lambda s: (s.find('Auto-disabling due to bad match')>=0), db['log_note'])))

    # New section - AEE
    # How many moves happened, according to the measured positions?
    sel = (obs_x!=0.0)|(obs_y!=0.0)&(exposure>0)
    dx = obs_x[sel][:-1]-obs_x[sel][1:]
    dy = obs_y[sel][:-1]-obs_y[sel][1:]
    moved = ((dx**2+dy**2)>0.05**2)
    Nmoved = np.sum(moved)
    if not quiet: print("Moved on the following exposures:", exposure[sel][:-1][moved])

    # New section - AEE
    # Look for mentions of other fibers
    pos_prog = re.compile('(M0\d\d\d\d)')
    def find_pos(log):
        match = pos_prog.search(log)
        if match: return match.groups()
    all_mentions = list(map(find_pos, db['log_note']))
    all_mentions = [item for i in all_mentions if i for item in i]
    mentions = np.unique(all_mentions)
    mentions = ' '.join(mentions)
    if not quiet: print("Other fibers mentioned in the log", mentions)
    
    # Now count the number of certain cases
    Nlines = len(db['ctrl_enabled'])
    Ndebounced = np.sum(debounced)
    Ndebounced_fail = np.sum(debounced_fail)
    Nautodisabled = np.sum(autodisabled)
    # New line - AEE
    Nenabled = np.sum(db['ctrl_enabled'])

    Ninterfere = np.sum(interfere)
    Nfrozen = np.sum(frozen)
    Nfrozen_unflagged = np.sum(frozen_unflagged)
    Nmoves = np.sum(move_attempted)
    Nbigmoves = np.sum(bigmove)
    
    # If a move yields a handle_fvc_feedback and the next move is denied and the previous move wasn't, that
    # may indicate that the fiber has come into contact with a neighbor, throwing off its
    # position.
    Nlodged = np.sum(fvc_feedback[1:-1]&denied[:-2]&denied[2:])

    if not quiet: 
        print("Returned", Nlines, "lines from the DB.")
        print("Found", Ndebounced,"cases of Debouncing.", Ninterfere,"cases of interference.", Nfrozen,"cases of being frozen.")
        print(Nmoves, "moves attempted.  Big moves:", Nbigmoves)
        #print(np.sum(fvc_feedback), "instances of handle_fvc_feedback")

    # We're concerned when fvc_feedback happens and the previous entry attempted a big move.
    # Alas, these vectors are 1 shorter than the original set.
    fvc_update = fvc_feedback[:-1] & bigmove[1:]
    moved_t = db['pos_t'][:-1]-(db['pos_t'][1:]-move_t[1:])
    moved_p = db['pos_p'][:-1]-(db['pos_p'][1:]-move_p[1:])
    scale_t = moved_t/(move_t[1:]+1e-10)
    scale_p = moved_p/(move_p[1:]+1e-10)


    fvc_noupdate = ~fvc_feedback[:-1] & bigmove[1:]
    Nbigmoves_ok = np.sum(fvc_noupdate)
    Nbigmoves_up = np.sum(fvc_update)

    bad_move_t = fvc_update & bigmove_t[1:] & (np.abs(scale_t-1.0)>0.1)
    bad_move_p = fvc_update & bigmove_p[1:] & (np.abs(scale_p-1.0)>0.1)

    Nbad_t = np.sum(bad_move_t)
    Nbad_p = np.sum(bad_move_p)
    if not quiet:
        print(Nbigmoves_ok, "big moves were considered ok by FVC.")
        print(Nbigmoves_up, "big moves were followed by FVC update and usable for scale measurements.")
        print("Found", Nbad_t, "theta and", Nbad_p,"phi bad moves with a scale factor more than 10% off")
        # This block is new - AEE
        try: last_bad_theta_date = db['time_recorded'][1:][bad_move_t][-1].isoformat()[0:10]
        except: last_bad_theta_date = ''
        print("Bad theta exposures:", exposure[1:][bad_move_t], last_bad_theta_date)
        try: last_bad_phi_date = db['time_recorded'][1:][bad_move_p][-1].isoformat()[0:10]
        except: last_bad_phi_date = ''
        print("Bad phi exposures:", exposure[1:][bad_move_p], last_bad_phi_date)

    tmp = scale_t[fvc_update&bigmove_t[1:]]
#     if not quiet:
#         print("moved_t", moved_t[fvc_update&bigmove_t[1:]])
#         print("move_t", move_t[1:][fvc_update&bigmove_t[1:]])
    
    if (len(tmp)>1):
        scale_t_rms = np.std(tmp)
        if (len(tmp)>2):
            tmp_clip = np.sort(tmp)
            if tmp_clip[-1]>1.3: tmp_clip = tmp_clip[0:-1]   # Clip the highest value if big
            scale_t_mean = np.mean(tmp_clip)
            scale_t_err = np.std(tmp_clip)/np.sqrt(len(tmp_clip)+1)
        else:
            scale_t_mean = np.mean(tmp)
            scale_t_err = np.std(tmp)/np.sqrt(len(tmp)+1)
        scale_t_max = np.max(np.abs(tmp-1.0))
    else:
        scale_t_mean = 0.0
        scale_t_rms = 0.0
        scale_t_err = 0.0
        scale_t_max = 0.0

    if not quiet: 
        print("Theta:", f'{scale_t_mean:6.4f}', "+-", f'{scale_t_rms:6.4f}'," max", scale_t_max)
        print(tmp)

    tmp = scale_p[fvc_update&bigmove_p[1:]]
    if (len(tmp)>1):
        scale_p_rms = np.std(tmp)
        if (len(tmp)>2):
            tmp_clip = np.sort(tmp)
            if tmp_clip[-1]>1.3: tmp_clip = tmp_clip[0:-1]   # Clip the highest value if big
            scale_p_mean = np.mean(tmp_clip)
            scale_p_err = np.std(tmp_clip)/np.sqrt(len(tmp_clip)+1)
        else:
            scale_p_mean = np.mean(tmp)
            scale_p_err = np.std(tmp)/np.sqrt(len(tmp)+1)
        scale_p_max = np.max(np.abs(tmp-1.0))
    else:
        scale_p_mean = 0.0
        scale_p_rms = 0.0    
        scale_p_err = 0.0    
        scale_p_max = 0.0
    if not quiet: 
        print("Phi:  ", f'{scale_p_mean:6.4f}', "+-", f'{scale_p_rms:6.4f}'," max", scale_p_max)
        print(tmp)

    # We can also look for cases where fvc_feedback keeps returning the same value.
    # These may be cases pinned against hardstops, for instance.
    # TODO: This requires all cases in a night to be within 1 deg rms.  But if we drive into 
    # such a point in the middle of the night, we might prefer to ask if the last 3-5 measurements
    # are all too close.
    hard_theta = 0.0
    hard_phi = 0.0
    if (np.sum(fvc_update)>3):
        tmp = db['pos_t'][:-1][fvc_update][:4]
        if (len(tmp)>2)&(np.std(tmp)<1): hard_theta = np.mean(tmp)
        tmp = db['pos_p'][:-1][fvc_update][:4]
        if (len(tmp)>2)&(np.std(tmp)<1): hard_phi = np.mean(tmp)        

    # The 4-digit Location is a helpful way to match to the coordinate files
    location = petal_id2loc[db['petal_id'][0]]*1000+db['device_loc']
    
    # This block is updated in multiple ways - AEE
    if not quiet and showlog: 
        redacted = db[('exposure_id','exposure_iter','pos_t','pos_p','log_note')]
        redacted['pos_t'].info.format = '7.3f'
        redacted['pos_p'].info.format = '6.3f'
        redacted['exposure_id'].info.name = 'expos'
        redacted['exposure_iter'].info.name = 'iter'
        redacted['log_note'].info.format = '<s'
        print(redacted.pprint_all())
        print()
    
    # This list is slightly longer - AEE
    # Now return this in a recarray, suitable for concatenation.
    return np.array([(db['pos_id'][0], db['petal_id'][0], location[0], mentions, Nautodisabled, Nlodged, Ndebounced, Ndebounced_fail, Ninterfere, Nfrozen, Nfrozen_unflagged, \
                      Nlines, Nenabled, Nmoves, Nmoved, Nbigmoves, Nbigmoves_ok, Nbigmoves_up, Nbad_t, Nbad_p, hard_theta, scale_t_mean, scale_t_err, scale_t_rms, scale_t_max, hard_phi, scale_p_mean, scale_p_err, scale_p_rms, scale_p_max)], \
                    dtype=[('pos_id','<U6'), \
                          ('petal_id','<i4'), \
                          ('location','<i4'), \
                          ('others','<S24'), \
                          ('Nautodis','i4'), \
                          ('Nlodged','i4'), \
                          ('Ndebounced','i4'), \
                          ('Ndebounced_fail','i4'), \
                          ('Ninterfere','<i4'), \
                          ('Nfrozen','<i4'), \
                          ('Nfrozen_unflag','<i4'), \
                          ('Nlines','<i4'), \
                          ('Nenabled','<i4'), \
                          ('Nmoves','<i4'), \
                          ('Nmoved','<i4'), \
                          ('Nbigmoves','<i4'), \
                          ('Nbigmoves_ok','<i4'), \
                          ('Nbigmoves_up','<i4'), \
                          ('Nbad_t','<i4'), \
                          ('Nbad_p','<i4'), \
                          ('hard_t','<f4'), \
                          ('scale_t_mean','<f4'), \
                          ('scale_t_err','<f4'), \
                          ('scale_t_rms','<f4'), \
                          ('scale_t_max','<f4'), \
                          ('hard_p','<f4'), \
                          ('scale_p_mean','<f4'), \
                          ('scale_p_err','<f4'), \
                          ('scale_p_rms','<f4'), \
                          ('scale_p_max','<f4') \
                          ])
    
#analyze_fiber(db)

In [12]:
# This block is new, or I'd removed it before - AEE
# Display the detailed results for a single fiber
def show_one_fiber(pos):
    onepos = analysis[(analysis["pos_id"]==pos)]
    print("Found", pos, "on petal", onepos["petal_id"][0])
    db = dbs[onepos["petal_id"][0]]
    analyze_fiber(db[(np.array(db['pos_id'])==pos)])
    return analysis[(analysis["pos_id"]==pos)]

In [13]:
# Here's how one would ask about a single fiber; note that we need the petal id too.
# TODO: might code a search across the petals....
#db = dbs[11]
#out = analyze_fiber(db[(np.array(db['pos_id'])=='M02866')])
#Table(out).show_in_notebook()

In [14]:
analysis = []

for petal in petal_id2loc.keys():
    print("Processing petal", petal)
    db = dbs[petal]
    poslist = np.unique(db['pos_id'])
    for pos in poslist:
        analysis.append(analyze_fiber(db[(np.array(db['pos_id'])==pos)], quiet=True))
    
# Adding this back in - AEE
print('All petals processed')

analysis = np.concatenate([x for x in analysis])
analysis = Table(analysis)
# Added by me - AEE
date_string = np.datetime64('%s-%s-%s' % (str(date_to_analyze)[0:4], str(date_to_analyze)[4:6], str(date_to_analyze)[6:]))
analysis.add_column(np.datetime64(date_string), name='date', index=0)
analysis['scale_t_mean'].info.format = '5.3f'
analysis['scale_t_rms'].info.format = '5.3f'
analysis['scale_p_mean'].info.format = '5.3f'
analysis['scale_p_rms'].info.format = '5.3f'

Processing petal 4
Processing petal 5
Processing petal 6
Processing petal 3
Processing petal 8
Processing petal 10
Processing petal 11
Processing petal 2
Processing petal 7
Processing petal 9
All petals processed


In [15]:
#analysis[0:10].show_in_notebook()

In [16]:
hardstop = (analysis['hard_t']!=0) | (analysis['hard_p']!=0) 
print("Positioners showing evidence of being stuck at one angle (hardstop?),", np.sum(hardstop), "in total")
print(list(analysis[hardstop]['pos_id']))
# I've changed this to be included in output - AEE
#analysis[hardstop].show_in_notebook()
analysis.add_column(MaskedColumn(data=hardstop, name='hardstop', dtype=bool, fill_value=False), index=4)

Positioners showing evidence of being stuck at one angle (hardstop?), 0 in total
[]


In [17]:
bad_theta = (analysis['Nbad_t']>2)&(analysis['scale_t_max']>0.1)&(np.abs(analysis['scale_t_mean']-1)>0.1) \
    &(np.abs(analysis['scale_t_mean']-1)>2.5*np.sqrt(analysis['scale_t_err']**2+0.04**2))
bad_phi = (analysis['Nbad_p']>2)&(analysis['scale_p_max']>0.1)&(np.abs(analysis['scale_p_mean']-1)>0.1) \
    &(np.abs(analysis['scale_p_mean']-1)>2.5*np.sqrt(analysis['scale_p_err']**2+0.04**2))

bad_scale = (bad_theta | bad_phi) & ~hardstop

# This requires at least 3 instances of handle_fvc_feedback on a big move.
    
print("Positioners showing non-unity scale factors,", np.sum(bad_scale), "in total")
print(list(analysis[bad_scale]['pos_id']))
# Changing this to include analysis in data - AEE
#analysis[bad_scale].show_in_notebook()
analysis.add_column(MaskedColumn(data=bad_scale, name='bad_scale', dtype=bool, fill_value=False), index=5)

Positioners showing non-unity scale factors, 0 in total
[]


In [18]:
lodged = (analysis['Nlodged']>0) 
print("Positioners showing evidence of becoming lodged (FVC feedback, then denied further moves),", np.sum(lodged), "in total")
print(list(analysis[lodged]['pos_id']))
# Another bit of including this in the output - AEE
#analysis[lodged].show_in_notebook()
analysis.add_column(MaskedColumn(data=lodged, name='lodged', dtype=bool, fill_value=False), index=6)

Positioners showing evidence of becoming lodged (FVC feedback, then denied further moves), 0 in total
[]


In [19]:
# This section is new - AEE
# Todo: Need to figure out how to exclude the initial FPsetup, so that we can be more sensitive
disabled_moving = (analysis['Nenabled']==0)&(analysis['Nmoved']>1)
print("Disabled Positioners showing evidence of being bumped,", np.sum(disabled_moving), "in total")
print(list(analysis[disabled_moving]['pos_id']))
print(list(analysis[disabled_moving]['Nmoved']))
print("Running the individual fiber will give the exposure number; then plot the configuration.")
# Adding this to the output - AEE
#analysis[disabled_moving].show_in_notebook()
analysis.add_column(MaskedColumn(data=disabled_moving, name='disabled_moving', dtype=bool, fill_value=False), index=7)

Disabled Positioners showing evidence of being bumped, 3 in total
['M04013', 'M08299', 'M07924']
[2, 2, 2]
Running the individual fiber will give the exposure number; then plot the configuration.


In [20]:
autodisabled = (analysis['Nautodis']>1) 
#  TODO: Consider changing this to >1, since the ==1 case may not be during FP setup.
print("Positioners that were autodisabled, probably during FP setup,", np.sum(autodisabled), "in total")
print(list(analysis[autodisabled]['pos_id']))
# Adding out output - AEE
# Index is higher because of previous addition
#analysis[autodisabled].show_in_notebook()
analysis.add_column(MaskedColumn(data=autodisabled, name='autodisabled', dtype=bool, fill_value=False), index=8)

Positioners that were autodisabled, probably during FP setup, 0 in total
[]


In [21]:
concern = (analysis['Ndebounced']>2) | (analysis['Ninterfere']>2) | (analysis['Ndebounced_fail']>2) 
concern = concern & ~bad_scale & ~lodged & ~autodisabled
print("Positioners showing other interferences or debounces,", np.sum(concern), "in total")
print(list(analysis[concern]['pos_id']))
# Adding to output, incrementing index - AEE
#analysis[concern].show_in_notebook()
analysis.add_column(MaskedColumn(data=concern, name='concern', dtype=bool, fill_value=False), index=9)

Positioners showing other interferences or debounces, 2 in total
['M05680', 'M01859']


In [22]:
# The numbers in this cutoff have changed - AEE
frozen = (analysis["Nfrozen"]>3)|(analysis["Nfrozen_unflag"]>1)
frozen = frozen & ~concern & ~bad_scale & ~lodged
print("Positioners showing unusual amounts of being otherwise frozen,", np.sum(frozen), "in total")
print(list(analysis[frozen]['pos_id']))
# Adding to output, incrementing index - AEE
#analysis[frozen].show_in_notebook()
analysis.add_column(MaskedColumn(data=frozen, name='frozen', dtype=bool, fill_value=False), index=10)

Positioners showing unusual amounts of being otherwise frozen, 10 in total
['M05288', 'M03612', 'M01362', 'M02866', 'M06238', 'M05389', 'M07643', 'M07655', 'M07752', 'M07864']


In [23]:
# I commented this out - AEE
#show_one_fiber('M01784').show_in_notebook()

----
Code below here is trying to look back over many nights at a single fiber.

In [24]:
# This is new - AEE
def get_one_fiber(pos, petal, start, end):
    _night = f'{start:08d}'
    _night = date(int(_night[0:4]),int(_night[4:6]),int(_night[6:8]))
    start = _night
    _night = f'{end:08d}'
    _night = date(int(_night[0:4]),int(_night[4:6]),int(_night[6:8]))
    end = _night
    print("Searching DB from", start.isoformat(), 'to', end.isoformat())
    cmd = "select * from posmovedb.positioner_moves_p"+f'{petal:1d}'+" where (ctrl_enabled=True) and (pos_id = '"+pos+"') and (time_recorded > '"+start.isoformat()+"') and (time_recorded < '"+end.isoformat()+"') order by time_recorded desc"
    print(cmd)
    db=dbquery(comm,cmd)
    db = Table(db)
    print(len(db), "entries retrieved")
    return db

In [25]:
#fib = get_one_fiber('M04915', 7, 20210410, 20210413)

In [26]:
#analyze_fiber(fib,quiet=False, showlog=True)

In [27]:
# This is all mine - AEE
if write_to_file:
    output_to_file(analysis, '%i_analysis.json' % date_to_analyze, format='pandas.json')
    output_to_file(analysis, '%i_analysis.csv' % date_to_analyze)
if write_to_db:
    # This is a real pain to do with numpy, but is a one liner on a dataframe
    from pandas import DataFrame
    from sqlalchemy import create_engine
    import urllib
    pandas_analysis = DataFrame(data=[row for row in analysis], columns=[c.lower() for c in list(analysis.columns)])
    # Ok! To the database!
    writer_args = {'host': os.getenv('DOS_DB_HOST'),
                   'port': int(os.getenv('DOS_DB_PORT')),
                   'database': os.getenv('DOS_DB_NAME'),
                   'user': os.getenv('DOS_DB_WRITER'),
                   'password': urllib.parse.quote_plus(os.getenv('DOS_DB_WRITER_PASSWORD'))}
    writer_engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**writer_args))
    #help(pandas_analysis.to_sql)
    pandas_analysis.to_sql(name='fiber_analysis', con=writer_engine, schema='telemetry', if_exists='append', index=False)
    print('Writing to DB complete')
end_time = time.time()
print('This took %f seconds' % (end_time - start_time))

Writing to DB complete
This took 67.188812 seconds


In [28]:
# I'm not sure why this is here? Is this instead of a sys.exit() that would end a script? - AEE
call_nonsense_to_halt_the_kernel()

NameError: name 'call_nonsense_to_halt_the_kernel' is not defined

------

Code below here is looking at the calibration database

In [ ]:
def get_calibDB():
    cmd = "select t.* from posmovedb.positioner_calibration t inner join ( select pos_id, max(time_recorded) as MaxDate from posmovedb.positioner_calibration group by pos_id ) tm on t.pos_id = tm.pos_id and t.time_recorded = tm.MaxDate"
    print(cmd)
    calib=dbquery(comm,cmd)
    calib = Table(calib)
    print(len(calib), "entries retrieved")
    return calib

In [ ]:
calib = get_calibDB()

In [ ]:
calib[:1].show_in_notebook()

In [ ]:
print(calib.dtype)

In [ ]:
buses = np.unique(calib['bus_id'])
for bus in buses:
    sel = (calib['bus_id']==bus)
    plt.scatter(calib['offset_x'][sel], calib['offset_y'][sel], s=12)

plt.title("Distinct CANbus Regions on a Petal", size=14)
plt.show()

In [ ]:
linear_phi = (calib["gear_calib_p"]<1)
linear_theta = (calib["gear_calib_t"]<1)

plt.hist(calib["offset_p"],bins=70,range=[-20,50])
plt.hist(calib["offset_p"][linear_phi],bins=70,range=[-20,50],color='r')
plt.xlabel("Offset_p", size=14)
plt.ylabel("Histogram (note log-scale)", size=14)
plt.text(15, 800, "Red: gear_calib_p<1", size=14)
plt.text(20, 400, "(aka 'linear-phi')", size=14)
plt.semilogy()
plt.show()

-------

Looking at the calibrations

In [ ]:
cal = Table.read("/global/u1/e/eisenste/Calibration/20201217/phi_theta_fits.csv")
dx = cal['phi_x']-cal['theta_x']
dy = cal['phi_y']-cal['theta_y']
theta_arm = np.sqrt(dx*dx+dy*dy)
good = (cal['phi_r']>0)&(cal['theta_r']>0)&(cal['phi_r']<3.5)
print(len(good), np.sum(good))

In [ ]:
print(np.median(cal['phi_r']), np.max(cal['phi_r']))
print(np.quantile(cal['phi_r'][good],[0.05,0.1,0.25,0.5,0.75,0.9,0.95]))


print(np.quantile(theta_arm[good],[0.05,0.1,0.25,0.5,0.75,0.9,0.95]))
print(np.sort(theta_arm[good]))

In [ ]:
from glob import glob
files = glob("/global/u1/e/eisenste/Calibration/20201217/dat/M*_paramfits.csv")
tabs = []
for file in files:
    tabs.append(Table.read(file))

In [ ]:
import astropy.table
params = astropy.table.vstack(tabs, join_type='outer')

In [ ]:
sel = (params['NUM_POINTS']>30)&(params['ERR.LENGTH_R1_STATIC']<0.5)&(params['ERR.LENGTH_R2_STATIC']<0.5)
params[sel]

In [ ]:
sel = (params['POS_ID']=='M06185')
for k in params.keys():
    if k[:2]=='ER' and k[-3]=='T': print(k, float(params[sel][k]))
params[sel]

In [ ]:
import scipy.linalg

In [ ]:
param = params[((params['POS_ID']=='M06185'))]
#param = params[((params['POS_ID']=='M07403'))]


#param = Table.read("/global/u1/e/eisenste/Calibration/20201217/M07407_paramfits.csv")


par = ['LENGTH_R1_STATIC', 'LENGTH_R2_STATIC', 'OFFSET_X_STATIC', 'OFFSET_Y_STATIC', 'OFFSET_T_STATIC', 'OFFSET_P_STATIC']
for p in par:
    print(p, float(param[p]), float(param['ERR.'+p]) )
    
cov = np.zeros([len(par),len(par)])
for i,p in enumerate(par):
    for j,q in enumerate(par):
        if i==j: cov[i][j] = param['ERR.'+p]**2
        else:
            try:
                cov[i][j] = param['COV.'+p+'.'+q]
            except:
                cov[i][j] = param['COV.'+q+'.'+p]
            cov[j][i] = cov[i][j]
#print(cov)
isq = 1/np.sqrt(np.diag(cov))   
rcov = ((cov*isq).T*isq).T
print(rcov)
print(scipy.linalg.eigh(rcov))